<a href="https://colab.research.google.com/github/vu-topics-in-big-data-2022/Project-Incident-Team2/blob/master/Table_Cleanup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# necessary imports and installations
!pip install boto3 pandas plotly
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from google.colab import files
import boto3, json

In [ ]:
# reading the merged table (Incidents, Traffic, Weather, Roads)
df = pd.read_csv('merged.csv')

In [ ]:
# dropping duplicate/unnecessary columns that I renamed as a result of merging
df = df.drop(columns=['county', 'month_1', 'hour_of_day_1', 'day_1', 'window_of_day_1', 'day_of_week_1', 'window_id_1'])

In [ ]:
# during several transformations, data also transformed types (int -> double), so we need to change it back
df['weekend_or_not'] = df['weekend_or_not'].fillna(0).astype(int)
df['response_time_sec'] = df['response_time_sec'].fillna(0).astype(int)
df['year'] = df['year'].fillna(0).astype(int)
df['month'] = df['month'].fillna(0).astype(int)
df['xdgroup'] = df['xdgroup'].fillna(0).astype(int)
df['hour_of_day'] = df['hour_of_day'].fillna(0).astype(int)
df['day'] = df['day'].fillna(0).astype(int)
df['window_of_day'] = df['window_of_day'].fillna(0).astype(int)
df['incident_id'] = df['incident_id'].fillna(0).astype(int)
df['segid'] = df['segid'].fillna(0).astype(int)
df['xd_id'] = df['xd_id'].fillna(0).astype(int)
df['xdsegid'] = df['xdsegid'].fillna(0).astype(int)

In [ ]:
table = pa.Table.from_pandas(df)
pq.write_table(table, 'everything_merged.parquet')

In [ ]:
# can locally download the results here
files.download('everything_merged.parquet')

In [ ]:
# or alternatively, can link to AWS to write to an S3 Bucket
credentials = {
    'region_name': AWS_REGION,
    'aws_access_key_id': AWS_ACCESS_KEY,
    'aws_secret_access_key': AWS_SECRET_KEY,
    'aws_session_token': AWS_SESSION_TOKEN
}
session = boto3.session.Session(**credentials)
s3 = session.client('s3')
s3_url = 's3://bigdata-seg/outputs/everything_merged.parquet'
df.to_parquet(s3_url)